In [8]:
#Use 
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.stats import linregress
from pprint import pprint
from geopy.geocoders import Nominatim

output_data_file = "output/clean_abandon_data.csv"


In [9]:
#Get the JSON from Newark
url = 'http://data.ci.newark.nj.us/datastore/odata3.0/796e2a01-d459-4574-9a48-23805fe0c3e0?$format=json'
repsonse = requests.get(url).json()
repsonse2 = repsonse['value']

#Build a DataFrame
df_part1 = pd.DataFrame.from_dict(repsonse2)
df_part1 = df_part1.rename(columns={"Vital Address":"Address"})
df_part1 = df_part1.rename(columns={"_id":"ID"})

#Add missing city and state keys
df_part1['City'] = "Newark"
df_part1['State'] = "NJ"

#reorder columns to make more sense
col = ['ID','Lot','Block','Open','Owner Name','Address','City', 'State','Most Recent Inspection']
df_part2 = df_part1[col]

In [10]:
#define variables
zip_code = []
indices = []

#Grab indicies for loopin'
indices = list(df_part2.index.values)

# initialize Nominatim API (geopy.geocoders)
geolocator = Nominatim(user_agent="geoapiExercises")

#Loop it
for index in indices:
    try:
        place = f"{df_part2['Address'][index]} {df_part2['City'][index]} {df_part2['State'][index]}"
        location = geolocator.geocode(place)
        data = location.raw
        loc_data = data['display_name'].split()
        zip_code.append(loc_data[-3].replace(",",""))        
    except:
        zip_code.append("NULL")

#Build the dictionary and related DataFrame        
zip_dict = {"Zip Code": zip_code}
zip_df = pd.DataFrame(zip_dict)


In [11]:
df_part3 = df_part2.join(zip_df, how='left')
col = ['ID','Lot','Block','Open','Owner Name','Address','City', 'State','Zip Code','Most Recent Inspection']
df_part3 = df_part3[col]
df_part3

,ID,Lot,Block,Open,Owner Name,Address,City,State,Zip Code,Most Recent Inspection
0,1,3.0,118.00,Vacant Only,LIP LIFE REALTY INC C/O BYUNG JUN,1011 BROAD ST,Newark,NJ,07102,2017-09-27T00:00:00
1,2,11.1,1183.01,Vacant Only,"JERONIMO,RICARDO L BIGALIA,TAOANE",27-29 SUMO VILLAGE CT,Newark,NJ,07114,2016-10-19T00:00:00
2,3,33.0,1185.01,Vacant Only,BALAN INVESTMENTS LLC,8-10 GOBLE ST,Newark,NJ,07114,2020-02-20T00:00:00
3,4,41.0,121.00,Abandoned,493 WASHINGTON ST LLC,493 WASHINGTON ST,Newark,NJ,07102,2016-10-27T00:00:00
4,5,54.0,123.00,Vacant Only,VULCAN PIONEERS OF N J INC,97-99 CLINTON AVE,Newark,NJ,07114,2020-01-30T00:00:00
...,...,...,...,...,...,...,...,...,...,...
495,496,11.0,3580.00,Vacant Only,"MCCALL, RUTH % COMM MTGE CO",10-12 GRANT AVE,Newark,NJ,07112,2020-01-08T00:00:00
496,497,16.0,3582.00,Vacant Only,CITY OF NEWARK,145 WATSON AVE,Newark,NJ,07112,2019-11-14T00:00:00
497,498,13.0,3583.00,Vacant Only,"FDF 3 LLC,",916 HUNTERDON ST,Newark,NJ,07112,2018-08-27T00:00:00
498,499,13.0,3583.00,Vacant Only,"FDF 3 LLC,",916 HUNTERDON ST,Newark,NJ,07112,2020-08-27T00:00:00


In [12]:
index_drop = df_part3[df_part3['Zip Code'] == "NULL"].index
clean_aband_df = df_part3.drop(index_drop, inplace=False)
clean_aband_df

,ID,Lot,Block,Open,Owner Name,Address,City,State,Zip Code,Most Recent Inspection
0,1,3.0,118.00,Vacant Only,LIP LIFE REALTY INC C/O BYUNG JUN,1011 BROAD ST,Newark,NJ,07102,2017-09-27T00:00:00
1,2,11.1,1183.01,Vacant Only,"JERONIMO,RICARDO L BIGALIA,TAOANE",27-29 SUMO VILLAGE CT,Newark,NJ,07114,2016-10-19T00:00:00
2,3,33.0,1185.01,Vacant Only,BALAN INVESTMENTS LLC,8-10 GOBLE ST,Newark,NJ,07114,2020-02-20T00:00:00
3,4,41.0,121.00,Abandoned,493 WASHINGTON ST LLC,493 WASHINGTON ST,Newark,NJ,07102,2016-10-27T00:00:00
4,5,54.0,123.00,Vacant Only,VULCAN PIONEERS OF N J INC,97-99 CLINTON AVE,Newark,NJ,07114,2020-01-30T00:00:00
...,...,...,...,...,...,...,...,...,...,...
495,496,11.0,3580.00,Vacant Only,"MCCALL, RUTH % COMM MTGE CO",10-12 GRANT AVE,Newark,NJ,07112,2020-01-08T00:00:00
496,497,16.0,3582.00,Vacant Only,CITY OF NEWARK,145 WATSON AVE,Newark,NJ,07112,2019-11-14T00:00:00
497,498,13.0,3583.00,Vacant Only,"FDF 3 LLC,",916 HUNTERDON ST,Newark,NJ,07112,2018-08-27T00:00:00
498,499,13.0,3583.00,Vacant Only,"FDF 3 LLC,",916 HUNTERDON ST,Newark,NJ,07112,2020-08-27T00:00:00


In [13]:
clean_aband_df.to_csv(output_data_file, index=False, header=True)